<a href="https://www.kaggle.com/code/phamhoang1909/extract-frame?scriptVersionId=257141462" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!git clone https://github.com/hein-nkhh/tri-modal-keyframe-extraction.git

Cloning into 'tri-modal-keyframe-extraction'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 51 (delta 4), reused 51 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 1.41 MiB | 9.66 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
%cd /kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction

/kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction


In [3]:
%%writefile /kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction/main.py
from feature_extraction.color_features import color_feature_extraction
from feature_extraction.image_features import image_feature_extraction
from feature_extraction.content_features import content_feature_extraction
from feature_extraction.cross_modal_features import feature_level_fusion
from clustering import cluster_frames
from frame_selection import candidate_frame_selection
import time

def main():

    start = time.time()

    # video_folder = "evaluation/"
    # dataset_name = "Summe"

    full_path = "/kaggle/input/sample-video"

    print("Feature Extraction is started...")

    print("Extracting Color Features...")
    color_feature_extraction.extract_color_features(full_path, frame_interval=1)

    print("Extracting Image Features...")
    image_feature_extraction.extract_image_features(full_path, frame_interval=1)

    print("Extracting Content Features...")
    content_feature_extraction.extract_content_features(full_path, frame_interval=1)

    print("Fusing the Features...")
    feature_level_fusion.fuse_features()

    print("Clustering the Fused Features...")
    cluster_frames.cluster(visualize=False)

    print("Selecting the Candidate Keyframes...")
    candidate_frame_selection.find_candidate_keyframes(full_path)

    end = time.time()
    print("Time passed:" + str(end-start))
if __name__ == '__main__':
    main()

Overwriting /kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction/main.py


In [4]:
# from multiprocessing import set_start_method
# try:
#     set_start_method("spawn")
# except RuntimeError:
#     pass

In [5]:
%%writefile /kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction/feature_extraction/content_features/content_feature_extraction.py
import os
import cv2
import torch
from PIL import Image
from tqdm import tqdm
import csv
from multiprocessing import Pool, set_start_method
from transformers import MllamaForConditionalGeneration, AutoProcessor
from transformers import BitsAndBytesConfig
from feature_extraction.content_features import content_embeddings


def get_videos(directory='../../test_videos'):
    """
    Retrieves video file paths from the specified directory.
    """
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv')

    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

    video_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(video_extensions):
                video_files.append(os.path.join(root, file))

    if not video_files:
        print(f"No video files found in '{directory}'. Please add videos to process.")

    return video_files


def  initialize_model():
    """
    Initializes the Llama 3.2-11B-Vision-Instruct model and processor with quantization.
    """
    model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

    config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    
    # Set quantization configuration
    # quantization_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_quant_type='nf4',
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_compute_dtype=torch.bfloat16
    # )

    # Load the model with quantization
    model = MllamaForConditionalGeneration.from_pretrained(
        model_id,
        # quantization_config=quantization_config,
        # low_cpu_mem_usage=True,
        # torch_dtype=torch.bfloat16,
        quantization_config=config,
        device_map="auto"
    )
    model.eval()

    # Compile the model for potential speed gains (requires Torch 2.0+)
    model = torch.compile(model, mode="default")

    # Load the processor
    processor = AutoProcessor.from_pretrained(model_id)

    return model, processor, torch.device('cuda')


def generate_description(frames, model, processor, device):
    """
    Generates content descriptions for a list of frames using the model.
    If the model returns a response indicating it does not see the image,
    we replace the description with a default placeholder.
    """
    # Convert each frame from BGR to RGB using slicing (usually faster than cv2.cvtColor)
    images = [Image.fromarray(frame[..., ::-1]) for frame in frames]

    # Since the prompt is identical for every image, compute it once:
    prompt = processor.apply_chat_template(
        [{"role": "user", "content": [{"type": "image"},
                                       {"type": "text", "text": "In one sentence, describe the visible content of this provided image: "}] }],
        add_generation_prompt=True
    )
    # Replicate the same prompt for each image (batch_size remains 1 for each call)
    input_texts = [prompt] * len(images)

    inputs = processor(
        images=images,
        text=input_texts,
        add_special_tokens=False,
        return_tensors="pt",
        padding=True
    ).to(device)

    prompt_lengths = inputs['input_ids'].ne(processor.tokenizer.pad_token_id).sum(dim=1)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            do_sample=False,  # greedy decoding
            temperature=None,
            top_p=None
        )

    descriptions = []
    unwanted_tokens = ["<|eot_id|>", "<|finetune_right_pad_id|>"]

    # Keywords that indicate the model did not "see" the image
    no_image_keywords = [
        "have access to an image",
        "no image provided",
        "no image",
        "not seeing",
        "image is blank",
        "share the image with me",
        "see an image",
        "access images",
        "access the images",
        "unable to view or describe",
        "not able to view the image",
        "to view or access images",
        "see the image",
        "Unfortunately",
    ]

    for i, output in enumerate(outputs):
        prompt_length = prompt_lengths[i]
        response = processor.decode(output[prompt_length:], skip_special_tokens=True)
        # Remove unwanted tokens manually
        for t in unwanted_tokens:
            response = response.replace(t, "")
        response = response.strip()

        if any(keyword in response.lower() for keyword in no_image_keywords):
            response = "No visible content"

        descriptions.append(response)

    return descriptions


def process_videos_on_gpu(args):
    """
    Processes the assigned videos on the given GPU.
    Uses a background thread to prefetch frames while running inference.
    """
    import threading
    from queue import Queue, Empty

    gpu_id, video_files, frame_interval = args

    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

    model, processor, device = initialize_model()

    all_results = []

    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        cap = cv2.VideoCapture(video_file)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        results = []

        pbar = tqdm(total=total_frames, desc=f"Processing {video_name} on GPU {gpu_id}", leave=False)

        # Queue to hold prefetched frames (adjust maxsize as needed)
        frame_queue = Queue(maxsize=20)
        stop_event = threading.Event()

        def read_frames():
            idx = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if idx % frame_interval == 0:
                    frame_queue.put((idx, frame))
                idx += 1
                # Update progress bar as each frame is read.
                pbar.update(1)
            stop_event.set()

        reader_thread = threading.Thread(target=read_frames)
        reader_thread.start()

        # Process frames as they become available.
        while not (stop_event.is_set() and frame_queue.empty()):
            try:
                frame_idx, frame = frame_queue.get(timeout=1)
            except Empty:
                continue
            try:
                # Process a single frame (batch size = 1)
                description = generate_description([frame], model, processor, device)[0]
                results.append({
                    'video_id': video_name,
                    'frame_id': frame_idx,
                    'description': description
                })
            except Exception as e:
                print(f"Error processing frame {frame_idx} of {video_name} on GPU {gpu_id}: {e}")

        reader_thread.join()
        pbar.close()
        cap.release()
        all_results.extend(results)

    del model
    torch.cuda.empty_cache()

    return all_results


def extract_content_features(video_path='../../test_videos', frame_interval=10):
    """
    Retrieves videos, processes them in parallel across available GPUs, and saves
    the frame descriptions to CSV. Finally, converts the descriptions to embeddings.
    """
    # Ensure multiprocessing uses 'spawn' method
    set_start_method('spawn', force=True)

    # Step 1: Get list of video files
    video_files = get_videos(video_path)
    if not video_files:
        return

    # Determine the number of available GPUs
    num_gpus = torch.cuda.device_count()

    if num_gpus == 0:
        print("No GPUs available. Exiting.")
        return

    # Distribute videos across GPUs
    video_chunks = [[] for _ in range(num_gpus)]
    for idx, video_file in enumerate(video_files):
        gpu_id = idx % num_gpus
        video_chunks[gpu_id].append(video_file)

    # Prepare arguments for multiprocessing
    args_list = []
    for gpu_id in range(num_gpus):
        args_list.append((gpu_id, video_chunks[gpu_id], frame_interval))

    # Use a multiprocessing Pool to process videos in parallel
    with Pool(processes=num_gpus) as pool:
        results = pool.map(process_videos_on_gpu, args_list)

    # Combine results from all GPUs
    all_results = []
    for gpu_results in results:
        all_results.extend(gpu_results)

    # Save the results to a CSV file
    output_csv = 'feature_extraction/content_features/content_descriptions.csv'
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['video_id', 'frame_id', 'description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for result in all_results:
            writer.writerow(result)

    print(f"Content descriptions saved to {output_csv}")

    print("Converting descriptions into embeddings...")
    content_embeddings.sentence_to_embeddings()


if __name__ == "__main__":
    extract_content_features()



Overwriting /kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction/feature_extraction/content_features/content_feature_extraction.py


In [6]:
!pwd

/kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction


In [7]:
!pip install -r /kaggle/working/tri-modal-keyframe-extraction/Keyframe_Extraction/requirements.txt

  Cloning https://github.com/christopherjenness/DBCV.git (to revision db7345c45ab7d33ea500778e5c9048fd9f7d3156) to /tmp/pip-install-0sxd5isq/dbcv_a98de2698f234e379c1dda4481990ae5
  Running command git clone --filter=blob:none --quiet https://github.com/christopherjenness/DBCV.git /tmp/pip-install-0sxd5isq/dbcv_a98de2698f234e379c1dda4481990ae5
  Running command git rev-parse -q --verify 'sha^db7345c45ab7d33ea500778e5c9048fd9f7d3156'
  Running command git fetch -q https://github.com/christopherjenness/DBCV.git db7345c45ab7d33ea500778e5c9048fd9f7d3156
  Resolved https://github.com/christopherjenness/DBCV.git to commit db7345c45ab7d33ea500778e5c9048fd9f7d3156
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a d

In [8]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [9]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUNGGING_FACE")

In [10]:
from huggingface_hub import login

# Dán token vào đây
login(secret_value_0)

In [11]:
!python main.py

2025-08-20 17:27:22.813888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755710843.164700     108 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755710843.263019     108 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Feature Extraction is started...
Extracting Color Features...
Processing Videos: 100%|██████████████████████| 1/1 [1:28:40<00:00, 5320.33s/it]
Features dictionary saved to feature_extraction/color_features/color_features_dict.h5
Extracting Image Features...
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*a